In [27]:
import pandas as pd
import numpy as np
import random as rd # generating random numbers
import datetime # manipulating date formats
from scipy import stats
from tqdm import tqdm_notebook
from itertools import product 


import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

from fbprophet import Prophet

from htsprophet.hts import hts, orderHier, makeWeekly
from htsprophet.htsPlot import plotNode, plotChild, plotNodeComponents


# settings
import warnings
import platform, sys
warnings.filterwarnings("ignore")

print('Machine: {} {}'.format(platform.uname().system, platform.uname().machine))
print("Python version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

pd.set_option('display.max_columns', 500)
pd.set_option("display.max_rows", 60)
pd.set_option('display.width', 1000)

Machine: Windows AMD64
Python version: 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Pandas version: 0.23.0


In [28]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
sales=pd.read_csv("D:\\all\\sales_train_v2.csv")
shops=pd.read_csv("D:\\all\\shops.csv")
item_cat=pd.read_csv("D:\\all\\item_categories.csv")
item=pd.read_csv("D:\\all\\items.csv")
sub=pd.read_csv("D:\\all\\sample_submission.csv")
test=pd.read_csv("D:\\all\\test.csv")

In [4]:
sales.item_price[sales.item_price<=0]

484683   -1.0
Name: item_price, dtype: float64

In [5]:
sales[(sales.item_id==2973) & (sales.shop_id==32) ]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
67427,29.01.2013,0,32,2973,2499.0,1.0
67428,25.01.2013,0,32,2973,2499.0,1.0
67429,22.01.2013,0,32,2973,2499.0,1.0
67430,21.01.2013,0,32,2973,2499.0,1.0
67431,18.01.2013,0,32,2973,2499.0,1.0
67432,17.01.2013,0,32,2973,2499.0,1.0
67433,15.01.2013,0,32,2973,2499.0,3.0
187844,05.02.2013,1,32,2973,2499.0,1.0
187845,14.02.2013,1,32,2973,2499.0,1.0
484682,23.05.2013,4,32,2973,1249.0,1.0


In [6]:
# correct negative price with the most frequent
sales.at[484683,"item_price"]=2499.0

In [10]:
#correct sales in 10th shop in 25 period - copy from 11th
sales.pivot_table(index="date_block_num", columns="shop_id", values="item_cnt_day", aggfunc="sum" )

shop_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
date_block_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5578.0,2947.0,1146.0,767.0,2114.0,NaN,3686.0,2495.0,1463.0,NaN,935.0,NaN,842.0,1178.0,1777.0,2724.0,2978.0,NaN,2856.0,3038.0,NaN,1495.0,1924.0,1948.0,1768.0,8215.0,2331.0,5548.0,7057.0,2820.0,2978.0,10173.0,1238.0,NaN,NaN,2127.0,NaN,2399.0,1704.0,NaN,NaN,1998.0,3991.0,1759.0,1708.0,1634.0,3363.0,2115.0,NaN,NaN,3406.0,2748.0,1812.0,2055.0,9386.0,NaN,3238.0,NaN,NaN,2017.0
1,6127.0,3364.0,488.0,798.0,2025.0,877.0,4007.0,2513.0,1156.0,NaN,1026.0,NaN,1209.0,1036.0,1795.0,2746.0,2614.0,NaN,3341.0,2834.0,NaN,1854.0,2447.0,1941.0,1782.0,7934.0,2597.0,4546.0,5438.0,2834.0,2717.0,11160.0,1494.0,NaN,NaN,2205.0,NaN,2216.0,1670.0,NaN,NaN,1856.0,3965.0,2033.0,1823.0,1530.0,2660.0,1824.0,NaN,NaN,3054.0,2284.0,1737.0,1906.0,8075.0,NaN,2655.0,NaN,NaN,1897.0
2,NaN,NaN,753.0,959.0,2060.0,1355.0,4519.0,2460.0,977.0,NaN,1017.0,NaN,1419.0,1245.0,1893.0,2835.0,3088.0,NaN,3551.0,3020.0,NaN,2233.0,2588.0,2118.0,2014.0,9732.0,3036.0,5454.0,6918.0,3151.0,3026.0,12670.0,1890.0,NaN,NaN,2568.0,NaN,2629.0,2009.0,NaN,NaN,1816.0,4953.0,2504.0,2131.0,1883.0,3103.0,1977.0,NaN,NaN,3610.0,2605.0,2034.0,2365.0,9488.0,NaN,3099.0,6586.0,3773.0,2028.0
3,NaN,NaN,583.0,647.0,285.0,1008.0,3168.0,1540.0,-1.0,NaN,756.0,NaN,1364.0,1018.0,1247.0,2168.0,2118.0,NaN,2421.0,2291.0,NaN,1846.0,2297.0,1698.0,787.0,7867.0,2381.0,3946.0,4910.0,2252.0,2379.0,9621.0,1364.0,NaN,NaN,1845.0,NaN,1946.0,1375.0,NaN,NaN,1387.0,3978.0,1880.0,1628.0,1253.0,2160.0,1540.0,NaN,NaN,2740.0,1945.0,1446.0,1515.0,6726.0,NaN,2153.0,5493.0,2831.0,1388.0
4,NaN,NaN,553.0,710.0,1211.0,1110.0,3022.0,1647.0,NaN,NaN,683.0,NaN,917.0,1104.0,1489.0,1931.0,2261.0,NaN,2944.0,2089.0,NaN,1646.0,1755.0,NaN,1967.0,6860.0,2148.0,3518.0,5703.0,2345.0,2304.0,9423.0,1209.0,NaN,NaN,2048.0,NaN,2019.0,1337.0,NaN,NaN,1597.0,3594.0,1624.0,1379.0,1333.0,2522.0,1724.0,NaN,NaN,2839.0,2243.0,1482.0,1767.0,7006.0,852.0,2140.0,5233.0,2308.0,1374.0
5,NaN,NaN,832.0,784.0,1464.0,1393.0,3847.0,2085.0,NaN,NaN,764.0,NaN,1710.0,1358.0,1863.0,2419.0,3075.0,906.0,3144.0,2638.0,NaN,1912.0,2243.0,NaN,2599.0,8478.0,2661.0,275.0,7056.0,2803.0,2794.0,10072.0,1493.0,NaN,NaN,2280.0,NaN,1829.0,1539.0,NaN,NaN,2185.0,4349.0,2060.0,1666.0,1785.0,2865.0,1654.0,NaN,NaN,3761.0,2200.0,1815.0,2125.0,8101.0,728.0,3105.0,5987.0,2972.0,1707.0
6,NaN,NaN,807.0,721.0,1378.0,1265.0,3360.0,2031.0,NaN,NaN,800.0,NaN,723.0,1094.0,1709.0,2599.0,2898.0,1044.0,2676.0,2459.0,NaN,1860.0,1903.0,NaN,2377.0,7361.0,2163.0,4148.0,6739.0,2205.0,2438.0,9500.0,93.0,NaN,NaN,2041.0,NaN,1293.0,1453.0,NaN,NaN,1997.0,3801.0,1632.0,1487.0,1507.0,2966.0,1329.0,NaN,NaN,3235.0,2273.0,1616.0,1975.0,7225.0,795.0,3294.0,5827.0,3122.0,1747.0
7,NaN,NaN,875.0,890.0,1713.0,1510.0,3702.0,2397.0,NaN,NaN,755.0,NaN,1599.0,944.0,2216.0,2536.0,2457.0,1306.0,1778.0,2675.0,NaN,2061.0,2017.0,NaN,2304.0,7784.0,2452.0,4925.0,7521.0,2073.0,2799.0,9882.0,NaN,NaN,NaN,2391.0,NaN,1472.0,1895.0,NaN,NaN,2149.0,4035.0,1849.0,1610.0,1649.0,3315.0,1610.0,NaN,NaN,3718.0,2142.0,1772.0,2367.0,7792.0,719.0,3311.0,5413.0,2863.0,2048.0
8,NaN,NaN,945.0,1026.0,1536.0,1298.0,4208.0,2599.0,NaN,NaN,1027.0,NaN,2032.0,1153.0,796.0,2790.0,1924.0,1459.0,2774.0,2801.0,NaN,2486.0,2518.0,NaN,3072.0,9251.0,3206.0,4981.0,6753.0,2832.0,3023.0,11204.0,NaN,NaN,NaN,2473.0,NaN,1666.0,1666.0,NaN,NaN,2146.0,5093.0,2768.0,1849.0,1725.0,2963.0,1742.0,NaN,NaN,3655.0,1903.0,2125.0,2440.0,7827.0,812.0,2829.0,5213.0,2735.0,2008.0


In [8]:
t2=sales[(sales["shop_id"]==11) & (sales["date_block_num"]==25)]
for i,j in t2.iterrows():
    t2.at[i, "shop_id"]=10


In [9]:
sales=pd.concat([sales,t2], ignore_index=True)
#sales[(sales["shop_id"]==10) & (sales["date_block_num"]==25)]

In [11]:
#add new feature - city
shops["city"]=shops["shop_name"].map(lambda x: str(x).split()[0])
shops.head()

,shop_name,shop_id,city
0,"!Якутск Орджоникидзе, 56 фран",0,!Якутск
1,"!Якутск ТЦ ""Центральный"" фран",1,!Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский


In [16]:
#shops.city.value_counts()

In [13]:
needed_shops=test["shop_id"].unique()
all_shops=range(0,59)
drop_shops=set(all_shops)-set(needed_shops)

In [18]:
shops1=shops.copy()

In [19]:
#del shops not in test sample
shops1.drop(drop_shops, axis=0, inplace=True)
#shops1.city.value_counts()

In [21]:
sales["date"]=sales["date"].apply(lambda x: datetime.datetime.strptime(x, '%d.%m.%Y'))

In [22]:
#add city column to sales data
sales=pd.merge(sales, shops[["shop_id","city"]], how="left", on="shop_id")

In [23]:
# remove cities which are not asked to predict
cities_remove=set(shops.city)-set(shops1.city)
indices_remove=[i for i, j in sales.iterrows() if j[6] in cities_remove]
sales1 = sales.drop(indices_remove, axis=0)

In [24]:
cities_remove

{'!Якутск', 'Выездная', 'Мытищи', 'Химки'}

In [26]:
# remove shops which are not asked to predict
indices_remove=[i for i, j in sales1.iterrows() if j[2] in drop_shops]
sales2 = sales1.drop(indices_remove, axis=0)

### by shop 

In [34]:
sales_by_shop=sales2.groupby([ "date_block_num", "shop_id" ])["item_cnt_day"].sum()
sales_by_shop=sales_by_shop.unstack(level=1)
sales_by_shop=sales_by_shop.fillna(0)
dates=pd.date_range(start = '2013-01-01',end='2015-10-01', freq = 'MS')
sales_by_shop.index=dates
sales_by_shop=sales_by_shop.reset_index()

In [35]:
sales_by_shop

shop_id,index,2,3,4,5,6,7,10,12,14,15,16,18,19,21,22,24,25,26,28,31,34,35,36,37,38,39,41,42,44,45,46,47,48,49,50,52,53,55,56,57,58,59
0,2013-01-01,1146.0,767.0,2114.0,0.0,3686.0,2495.0,935.0,842.0,1777.0,2724.0,2978.0,2856.0,3038.0,1495.0,1924.0,1768.0,8215.0,2331.0,7057.0,10173.0,0.0,2127.0,0.0,2399.0,1704.0,0.0,1998.0,3991.0,1708.0,1634.0,3363.0,2115.0,0.0,0.0,3406.0,1812.0,2055.0,0.0,3238.0,0.0,0.0,2017.0
1,2013-02-01,488.0,798.0,2025.0,877.0,4007.0,2513.0,1026.0,1209.0,1795.0,2746.0,2614.0,3341.0,2834.0,1854.0,2447.0,1782.0,7934.0,2597.0,5438.0,11160.0,0.0,2205.0,0.0,2216.0,1670.0,0.0,1856.0,3965.0,1823.0,1530.0,2660.0,1824.0,0.0,0.0,3054.0,1737.0,1906.0,0.0,2655.0,0.0,0.0,1897.0
2,2013-03-01,753.0,959.0,2060.0,1355.0,4519.0,2460.0,1017.0,1419.0,1893.0,2835.0,3088.0,3551.0,3020.0,2233.0,2588.0,2014.0,9732.0,3036.0,6918.0,12670.0,0.0,2568.0,0.0,2629.0,2009.0,0.0,1816.0,4953.0,2131.0,1883.0,3103.0,1977.0,0.0,0.0,3610.0,2034.0,2365.0,0.0,3099.0,6586.0,3773.0,2028.0
3,2013-04-01,583.0,647.0,285.0,1008.0,3168.0,1540.0,756.0,1364.0,1247.0,2168.0,2118.0,2421.0,2291.0,1846.0,2297.0,787.0,7867.0,2381.0,4910.0,9621.0,0.0,1845.0,0.0,1946.0,1375.0,0.0,1387.0,3978.0,1628.0,1253.0,2160.0,1540.0,0.0,0.0,2740.0,1446.0,1515.0,0.0,2153.0,5493.0,2831.0,1388.0
4,2013-05-01,553.0,710.0,1211.0,1110.0,3022.0,1647.0,683.0,917.0,1489.0,1931.0,2261.0,2944.0,2089.0,1646.0,1755.0,1967.0,6860.0,2148.0,5703.0,9423.0,0.0,2048.0,0.0,2019.0,1337.0,0.0,1597.0,3594.0,1379.0,1333.0,2522.0,1724.0,0.0,0.0,2839.0,1482.0,1767.0,852.0,2140.0,5233.0,2308.0,1374.0
5,2013-06-01,832.0,784.0,1464.0,1393.0,3847.0,2085.0,764.0,1710.0,1863.0,2419.0,3075.0,3144.0,2638.0,1912.0,2243.0,2599.0,8478.0,2661.0,7056.0,10072.0,0.0,2280.0,0.0,1829.0,1539.0,0.0,2185.0,4349.0,1666.0,1785.0,2865.0,1654.0,0.0,0.0,3761.0,1815.0,2125.0,728.0,3105.0,5987.0,2972.0,1707.0
6,2013-07-01,807.0,721.0,1378.0,1265.0,3360.0,2031.0,800.0,723.0,1709.0,2599.0,2898.0,2676.0,2459.0,1860.0,1903.0,2377.0,7361.0,2163.0,6739.0,9500.0,0.0,2041.0,0.0,1293.0,1453.0,0.0,1997.0,3801.0,1487.0,1507.0,2966.0,1329.0,0.0,0.0,3235.0,1616.0,1975.0,795.0,3294.0,5827.0,3122.0,1747.0
7,2013-08-01,875.0,890.0,1713.0,1510.0,3702.0,2397.0,755.0,1599.0,2216.0,2536.0,2457.0,1778.0,2675.0,2061.0,2017.0,2304.0,7784.0,2452.0,7521.0,9882.0,0.0,2391.0,0.0,1472.0,1895.0,0.0,2149.0,4035.0,1610.0,1649.0,3315.0,1610.0,0.0,0.0,3718.0,1772.0,2367.0,719.0,3311.0,5413.0,2863.0,2048.0
8,2013-09-01,945.0,1026.0,1536.0,1298.0,4208.0,2599.0,1027.0,2032.0,796.0,2790.0,1924.0,2774.0,2801.0,2486.0,2518.0,3072.0,9251.0,3206.0,6753.0,11204.0,0.0,2473.0,0.0,1666.0,1666.0,0.0,2146.0,5093.0,1849.0,1725.0,2963.0,1742.0,0.0,0.0,3655.0,2125.0,2440.0,812.0,2829.0,5213.0,2735.0,2008.0
9,2013-10-01,795.0,793.0,1482.0,1267.0,3675.0,2236.0,1013.0,1890.0,1534.0,2612.0,1800.0,2145.0,2499.0,2015.0,2364.0,2654.0,8356.0,2441.0,5609.0,10754.0,0.0,2083.0,0.0,1169.0,1519.0,0.0,2061.0,4464.0,1678.0,1593.0,2794.0,1647.0,0.0,0.0,3525.0,1965.0,2245.0,916.0,2375.0,5686.0,3022.0,1751.0


In [36]:
%%time
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

    

forecastsDict = {}
errors={}
for node in tqdm_notebook(range(sales_by_shop.shape[1]-1)):
    nodeToForecast = pd.concat([sales_by_shop.iloc[:,0], sales_by_shop.iloc[:, node+1]], axis = 1)
    nodeToForecast = nodeToForecast.rename(columns = {"index" : 'ds'})
    nodeToForecast["y"] = nodeToForecast[nodeToForecast.columns[1]]
    nodeToForecast.drop(nodeToForecast.columns[1], axis=1, inplace=True)
    growth = 'linear'
    m = Prophet(growth, yearly_seasonality=True)
    
     #  try-except because on some combinations model fails to converge
    try:
        m.fit(nodeToForecast)
        future = m.make_future_dataframe(periods = 1, freq = 'MS')
        forecast=m.predict(future)
    except:
        continue

    forecastsDict[sales_by_shop.columns.values[node+1]] = forecast
    errors[sales_by_shop.columns.values[node+1]]=mean_absolute_percentage_error(sales_by_shop.iloc[:, node+1],
                                                                             forecast.iloc[:-1,-1])
    #if (node== 50):
        #print("ready")
        #break




Wall time: 45.9 s


In [37]:
errors

{2: 11.5598174267394,
 3: 3.4389675792478647,
 4: 17.169233560604873,
 5: inf,
 6: 2.750518253570224,
 7: 2.860075653107572,
 10: 5.0618472658120774,
 12: 23.330460701393033,
 14: 10.564844564924654,
 15: 4.0092603513533565,
 16: 11.759172743082724,
 18: 16.57287022185884,
 19: 2.5556296021697222,
 21: 3.974290564461833,
 22: 6.697583353090487,
 24: 16.80222915002647,
 25: 3.881495585941212,
 26: 7.837171040221485,
 28: 7.1949490829110685,
 31: 1.8088107005997933,
 34: inf,
 35: 3.0377898416844107,
 36: inf,
 37: 15.364423775452114,
 38: 5.578139552135691,
 39: inf,
 41: 13.26527315323791,
 42: 3.2660099386761523,
 44: 5.096627801199667,
 45: 4.166566969085903,
 46: 5.204860978938123,
 47: 2.547266270455515,
 48: inf,
 49: inf,
 50: 17.569364073445406,
 52: 5.252164247255632,
 53: 1.536276543789179,
 55: inf,
 56: 4.053686155191946,
 57: inf,
 58: inf,
 59: 7.292804323597822}

In [39]:
#check why errors are so diverse
#t5=pd.crosstab(sales2.shop_id, sales2.date_block_num, values=sales2["item_cnt_day"],
 #           margins=True, normalize=True, aggfunc='std')

In [40]:
#t5.iloc[:, -1]
#sales2.groupby("shop_id")["item_cnt_day"].agg(["mean", "std"])

### hts trial

In [41]:
sales2.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,city
0,2013-01-02,0,59,22154,999.00,1.0,Ярославль
1,2013-01-03,0,25,2552,899.00,1.0,Москва
2,2013-01-05,0,25,2552,899.00,-1.0,Москва
3,2013-01-06,0,25,2554,1709.05,1.0,Москва
4,2013-01-15,0,25,2555,1099.00,1.0,Москва


In [43]:
hts_sales=sales2.drop(["date_block_num", "item_price", "city"], axis=1)
counts=hts_sales.item_cnt_day
hts_sales.drop(["item_cnt_day"], axis=1, inplace=True)
hts_sales["item_cnt_day"]=counts
hts_sales.head()

,date,shop_id,item_id,item_cnt_day
0,2013-01-02,59,22154,1.0
1,2013-01-03,25,2552,1.0
2,2013-01-05,25,2552,-1.0
3,2013-01-06,25,2554,1.0
4,2013-01-15,25,2555,1.0


In [44]:
def makeMonthly(data):
    columnList = data.columns.tolist()
    columnCount = len(columnList)-2
    if columnCount < 1:
        sys.exit("you need at least 1 column")
    data[columnList[0]] = pd.to_datetime(data[columnList[0]])
    cl = tuple(columnList[1:-1])
    data1 = data.groupby([pd.Grouper(key = columnList[0], freq='M'), *cl], as_index = False)[columnList[-1]].sum()
    data2 = data.groupby([pd.Grouper(key = columnList[0], freq='M'), *cl])[columnList[-1]].sum()
    data1['month'] = data2.index.get_level_values(columnList[0])
    cols = data1.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    data1 = data1[cols]
    return data1

In [45]:
hts_sales=makeMonthly(hts_sales)

In [46]:
hts_sales.shop_id=hts_sales.shop_id.astype('str')
hts_sales.item_id=hts_sales.item_id.astype('str')

In [48]:
hts_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1324166 entries, 0 to 1324165
Data columns (total 4 columns):
month           1324166 non-null datetime64[ns]
shop_id         1324166 non-null object
item_id         1324166 non-null object
item_cnt_day    1324166 non-null float64
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 50.5+ MB


In [58]:
hts_sales.head()

,month,shop_id,item_id,item_cnt_day
0,2013-01-31,2,27,1.0
1,2013-01-31,2,33,1.0
2,2013-01-31,2,317,1.0
3,2013-01-31,2,438,1.0
4,2013-01-31,2,471,2.0


In [59]:
hts_sales.shape

(1324166, 4)

In [61]:
# for 1000 sample time is 3.13 sec...for 1.3M should be about 1.5 hour
sample=hts_sales.loc[:10000, :]
sample.shape

(10001, 4)

In [63]:
%%time
data, nodes=orderHier(hts_sales, 1, 2)

ValueError: You are trying to merge on datetime64[ns] and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
def orderHier(data, col1 = 1, col2 = None, col3 = None, col4 = None, rmZeros = False):
    # 
    #This function will order the hierarchy the way you like it as long as you are
    #using max 4 layers
    #
    #
    #Inputs and desc:
    # 
    # Data - (pandas DataFrame) the data you want made into a hierarchical model
    #                           The first column should specify the time
    #                           The middle columns should be the names of the layers of the hierarchy (ex. Medium, Platform, BusinessMarket, etc.)
    #                           The last column should be the numeric column that you would like to forecast
    #
    # col1 - (int [1-4]) what layer you want the first column (thats not a timestamp) to be at
    #           Layers:
    #               1 - level right below total
    #               2 - level below 1
    #               3 - level below 2
    #               4 - Bottom Level
    #
    # col2 - (int [1-4]) what layer you want the second column (thats not a timestamp) to be at
    #           Layers:
    #               1 - level right below total
    #               2 - level below 1
    #               3 - level below 2
    #               4 - Bottom Level
    #
    # col3 - (int [1-4]) what layer you want the third column (thats not a timestamp) to be at
    #           Layers:
    #               1 - level right below total
    #               2 - level below 1
    #               3 - level below 2
    #               4 - Bottom Level
    #
    # 
    # col4 - (int [1-4]) what layer you want the fourth column (thats not a timestamp) to be at
    #           Layers:
    #               1 - level right below total
    #               2 - level below 1
    #               3 - level below 2
    #               4 - Bottom Level
    #
    #
    # Outputs and desc:
    # 
    # y - (pandas Dataframe) the DataFrame in a format that is consistent with the
    #                        Hierarchy function and ordered in a way that the user
    #                         specified.
    #
    #
    if col1 not in [1,2,3,4]:
        sys.exit("col1 should equal 1, 2, 3, or 4")
    if col2 is not None and col2 not in [1,2,3,4]:
        sys.exit("col3 should equal 1, 2, 3, or 4")
    if col3 is not None and col3 not in [1,2,3,4]:
        sys.exit("col3 should equal 1, 2, 3, or 4")
    if col1 is None:
        sys.exit("You need at least 1 column specified")
    if col2 is None:
        orderList = [col1]
        dimList = [data.columns.tolist()[1]]
        uniqueList = [data.iloc[:,1].unique()]
        lengthList = [len(uniqueList[0])]
        numIn = 1
    elif col3 is None:
        if col1 == col2:
            sys.exit("col1, col2 should all have different values")
        orderList = [col1, col2]
        dimList = [data.columns.tolist()[1], data.columns.tolist()[2]]
        uniqueList = [data.iloc[:,1].unique(), data.iloc[:,2].unique()]
        lengthList = [len(uniqueList[0]), len(uniqueList[1])]
        numIn = 2
    elif col4 is None:
        if col1 == col2 or col1 == col3 or col2 == col3:
            sys.exit("col1, col2, and col3 should all have different values")
        orderList = [col1,col2,col3]
        dimList = [data.columns.tolist()[1],data.columns.tolist()[2],data.columns.tolist()[3]]
        uniqueList = [data.iloc[:,1].unique(), data.iloc[:,2].unique(), data.iloc[:,3].unique()]
        lengthList = [len(uniqueList[0]), len(uniqueList[1]), len(uniqueList[2])]
        numIn = 3
    else:
        if col1 == col2 or col1 == col3 or col2 == col3 or col1 == col4 or col2 == col4 or col3 == col4:
            sys.exit("col1, col2, col3, and col4 should all have different values")
        orderList = [col1,col2,col3,col4]
        dimList = [data.columns.tolist()[1],data.columns.tolist()[2],data.columns.tolist()[3],data.columns.tolist()[4]]
        uniqueList = [data.iloc[:,1].unique(), data.iloc[:,2].unique(), data.iloc[:,3].unique(), data.iloc[:,4].unique()]
        lengthList = [len(uniqueList[0]), len(uniqueList[1]), len(uniqueList[2]), len(uniqueList[3])]
        numIn = 4
    
    numCol = data.columns.tolist()[-1]
    timeInterval = data.columns.tolist()[0]
    
    allDataframes = {}
    
    #Creating dataframes for the top level of the hierarchy (not total)
    for num in range(lengthList[orderList.index(1)]):
        allDataframes[uniqueList[orderList.index(1)][num]] = data.loc[data[dimList[orderList.index(1)]] == uniqueList[orderList.index(1)][num]]
		
        allDataframes[uniqueList[orderList.index(1)][num]+'1'] = (allDataframes[uniqueList[orderList.index(1)][num]].groupby([timeInterval])[numCol].sum()).to_frame()
        
        if numIn > 1:
            #Creating dataframes for the second level of the hierarchy
            placeholder = allDataframes[uniqueList[orderList.index(1)][num]].groupby([timeInterval, dimList[orderList.index(2)]])[numCol].sum()
            for ind in range(lengthList[orderList.index(2)]):
                allDataframes[uniqueList[orderList.index(1)][num]+'_'+uniqueList[orderList.index(2)][ind]] = (placeholder.loc[(placeholder.index.get_level_values(1) == uniqueList[orderList.index(2)][ind])]).to_frame()
                
                if numIn > 2:
                    placeholder1 = allDataframes[uniqueList[orderList.index(1)][num]].groupby([timeInterval, dimList[orderList.index(2)], dimList[orderList.index(3)]])[numCol].sum()
                    #Creating dataframes for the third level of the hierarchy
                    for cnt in range(lengthList[orderList.index(3)]):
                        allDataframes[uniqueList[orderList.index(1)][num]+'_'+uniqueList[orderList.index(2)][ind]+'_'+uniqueList[orderList.index(3)][cnt]] = (placeholder1.loc[(placeholder1.index.get_level_values(1) == uniqueList[orderList.index(2)][ind]) & (placeholder1.index.get_level_values(2) == uniqueList[orderList.index(3)][cnt])]).to_frame()
    
                        if numIn > 3:
                            placeholder2 = allDataframes[uniqueList[orderList.index(1)][num]].groupby([timeInterval, dimList[orderList.index(2)], dimList[orderList.index(3)], dimList[orderList.index(4)]])[numCol].sum()
                            #Creating dataframes for the third level of the hierarchy
                            for pos in range(lengthList[orderList.index(4)]):
                                allDataframes[uniqueList[orderList.index(1)][num]+'_'+uniqueList[orderList.index(2)][ind]+'_'+uniqueList[orderList.index(3)][cnt]+'_'+uniqueList[orderList.index(4)][pos]] = (placeholder2.loc[(placeholder2.index.get_level_values(1)\
                                               == uniqueList[orderList.index(2)][ind]) & (placeholder2.index.get_level_values(2) == uniqueList[orderList.index(3)][cnt]) & (placeholder2.index.get_level_values(3) == uniqueList[orderList.index(4)][pos])]).to_frame()
    
    #Creating total DataFrame
    allDataframes['Total'] = (data.groupby([timeInterval])[numCol].sum()).to_frame()

    #Take the sessions column from all those dataframes and put them into y
    y = pd.DataFrame(data = data[timeInterval].unique(), index = range(len(data[timeInterval].unique())), columns = ['time'])
    y = pd.merge(y, allDataframes['Total'], left_on = 'time', right_index = True)
    y.rename(columns = {numCol:'Total'}, inplace = True)
    
    for i in range(lengthList[orderList.index(1)]):
        y = pd.merge(y, allDataframes[uniqueList[orderList.index(1)][i]+'1'], how = 'left', left_on = 'time', right_index = True)
        y.rename(columns = {numCol:uniqueList[orderList.index(1)][i]}, inplace = True)
        if numIn > 1:
            for j in range(lengthList[orderList.index(2)]):
                allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]].index = allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]].index.droplevel(1)
                y = pd.merge(y, allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]], how = 'left', left_on = 'time', right_index = True)
                y.rename(columns = {numCol:uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]}, inplace = True)
                if numIn > 2:
                    for k in range(lengthList[orderList.index(3)]):
                        allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]].index = allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]].index.droplevel(2)
                        allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]].index = allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]].index.droplevel(1)
                        y = pd.merge(y, allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]], how = 'left', left_on = 'time', right_index = True)
                        y.rename(columns = {numCol:uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]}, inplace = True)
                        if numIn > 3:
                            for l in range(lengthList[orderList.index(4)]):
                                allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]+'_'+uniqueList[orderList.index(4)][l]].index = allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]+'_'+uniqueList[orderList.index(4)][l]].index.droplevel(3)
                                allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]+'_'+uniqueList[orderList.index(4)][l]].index = allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]+'_'+uniqueList[orderList.index(4)][l]].index.droplevel(2)
                                allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]+'_'+uniqueList[orderList.index(4)][l]].index = allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]+'_'+uniqueList[orderList.index(4)][l]].index.droplevel(1)
                                y = pd.merge(y, allDataframes[uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]+'_'+uniqueList[orderList.index(4)][l]], how = 'left', left_on = 'time', right_index = True)
                                y.rename(columns = {numCol:uniqueList[orderList.index(1)][i]+'_'+uniqueList[orderList.index(2)][j]+'_'+uniqueList[orderList.index(3)][k]+'_'+uniqueList[orderList.index(4)][l]}, inplace = True)
    
    
    if rmZeros == True:
        #Get rid of Missing columns and rows
        y.dropna(axis = 1, how = 'any', thresh = len(y['time'])/2, inplace = True)
        y.dropna(axis = 0, how = 'any', inplace = True)
    else:
        y = y.fillna(1)
    #Re-order the columns so that it is compatible with Hierarchy function
    cols = y.columns.tolist()
    list1 = []
    list2 = []
    list3 = []
    list4 = []
    for col in cols:
        if col.count('_') == 0:
            list1.append(col)
        if col.count('_') == 1:
            list2.append(col)
        if col.count('_') == 2:
            list3.append(col)
        if col.count('_') == 3:
            list4.append(col)
    
    newOrder = []
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    for item in list1:
        newOrder.append(item)
        count1 += 1
    for item in list2:
        newOrder.append(item)
        count2 += 1
    for item in list3:
        newOrder.append(item)
        count3 += 1
    for item in list4:
        newOrder.append(item)
        count4 += 1
        
    y = y[newOrder]
    
    ##
    # Create Nodes variable (A list of lists that describes the hierarchical structure)
    ##
    nodes = []
    nodes.append([count1-2])
    if numIn > 1:
        numberList = []
        for column in range(2, count1):
            number = sum([i.count(y.columns.tolist()[column]) for i in y.columns.tolist()[count1:count1+count2]])
            numberList.append(number)
        nodes.append(numberList)
    if numIn > 2:
        numberList = []
        for column in range(count1, count1+count2):
            number = sum([i.count(y.columns.tolist()[column]) for i in y.columns.tolist()[count1+count2:count1+count2+count3]])
            numberList.append(number)
        nodes.append(numberList)
    if numIn > 3:
        numberList = []
        for column in range(count1+count2, count1+count2+count3):
            number = sum([i.count(y.columns.tolist()[column]) for i in y.columns.tolist()[count1+count2+count3:count1+count2+count3+count4]])
            numberList.append(number)
        nodes.append(numberList)

    return y, nodes